In [ ]:
import math
import random
from collections import defaultdict, Counter, deque
from typing import Tuple, List, Set, Dict, Counter
from typing import Any, Sequence, Mapping, Iterable, Iterator
from itertools import product, chain, islice
import doctest
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import colorama
from colorama import Fore

In [ ]:
#change this for different models 

maxelogain = 32
number_players = 100 #must be even
games= 100
monte_n = 100

In [ ]:
class Player:
    """A player with real skill and obvservable elo.
    
    >>> Player('p1', '', '1000')  # test robustness
    Traceback (most recent call last):
        ...
    AssertionError: Must not be empty
    >>> playerA = Player('p1',1001, 1000)
    >>> playerA  # also tests __repr__()
    Player(1001, 1000)
    >>> playerA.update(10)
    >>> print(playerA)  # also tests __str__()
    (1001,1010)
    """
    def __init__(self,name:str, skill: int, elo:int) -> None:
        """Create player with given state.
        """
        assert name and skill and elo, "Must not be empty"
        self.skill: int = skill
        self.elo: int = elo
        self.name: str = name
            
    def __repr__(self) -> str:
        """Return machine-processable string representation of current state.
        """
        return f"Player({self.name},{str(self.skill)}, {str(self.elo)})"
    
    def __str__(self) -> str:
        """Return human-readable string representation of current state.
        """
        return f"({self.name},{self.skill},{self.elo})"
    
    def update(self, gain : int) -> None:
        """ Update the elo after a match"""
        self.elo = self.elo + gain

In [ ]:
#doctest.run_docstring_examples(Player, globals(), verbose=True, name="Player")

In [ ]:
def match_chess(player1 : Player, player2: Player)->None:
    P1win = 1/(1+10**((player2.skill - player1.skill)/400))
    P1expec = 1/(1+10**((player2.elo - player1.elo)/400))
    res = random.random()
    
    if res <= P1win:
        player1.update( int(maxelogain*(1-P1expec)) ) 
        player2.update( int(maxelogain*(0-1*(1-P1expec) )))
        print('match: ' + player1.name + ' vs '+player2.name+': '+player1.name + ' wins')
    elif res > P1win:
        player1.update( int(maxelogain*(0-P1expec) ) )
        player2.update( int(maxelogain*(1-1*(1-P1expec) )))
        print('match: ' + player1.name + ' vs '+player2.name+': '+player2.name + ' wins')

In [ ]:
def ranked(players:List[Player]):
    players2=players.copy()
    elos = [i.elo for i in players2]
    elos.sort()
    for i in range(len(elos)):
        elo = elos[i]
        for p in players2:
            if p.elo == elo:
                elos[i]=p
                players2.remove(p)
                break
    return elos

In [ ]:
def ranked_true(players:List[Player]):
    players2=players.copy()
    elos = [i.skill for i in players2]
    elos.sort()
    for i in range(len(elos)):
        elo = elos[i]
        for p in players2:
            if p.skill == elo:
                elos[i]=p
                players2.remove(p)
                break
    return elos

In [ ]:
def rand_matchmaking(players:List[Player]) -> List[Player]:
    random.shuffle(players)
    return players
        

In [ ]:
def chess_round_rand(players:List[Player]) -> None:
    order = rand_matchmaking(players)
    for i in range(len(players))[::2]:
        match_chess(order[i], order[i+1])

In [ ]:
def state(players:List[Player]):
    players2 = players.copy()
    state= [[i.name,i.skill, i.elo] for i in players2]
    return state

In [ ]:
def chess_game_rand(players:List[Player], games : int) -> List[List[Player]]:
    global Match_history 
    Match_history = [i for i in range(games+1)]
    for i in range(games):
        Match_history[i] = state(players)
        print('Round'+str(i))
        chess_round_rand(players)
    Match_history[-1] = state(players)
    return Match_history

In [ ]:
def Conv(lst:List[int], n:int)-> List[int]:
    conv = [lst[0]] + [(1/3)*(lst[i-1]+lst[i]+lst[i+1]) for i in range(1, n-1)] + [lst[n-1]]
    return conv

In [ ]:
def player_hist(player: str, match_hist):
    elos = []
    for state in match_hist:
        for p in state:
            if p[0] == player:  
                elos += [p[2]]
    return elos

In [ ]:
def summ_i(all_the_lines: List[List[int]], x:int)->float:
    all_is = [all_the_lines[i][x] for i in range(monte_n)]
    return sum(all_is)/len(all_is)

In [ ]:
def Sort_mcrlo(sub_li):
    sub_li.sort(key = lambda x: x[1])
    return sub_li
def elo_matchmaking(players:List[Player]) -> List[Player]:
    players.sort(key= lambda x: x.elo)
    return players
def chess_round_elo(players:List[Player]) -> None:
    order = elo_matchmaking(players)
    for i in range(len(players))[::2]:
        match_chess(order[i], order[i+1])    
def chess_game_elo(players:List[Player], games : int) -> List[List[Player]]:
    global Match_history 
    Match_history = [i for i in range(games+1)]
    for i in range(games):
        Match_history[i] = state(players)
        chess_round_elo(players)
    Match_history[-1] = state(players)
    return Match_history

In [ ]:
player_names = ['p'+str(i) for i in range(1,number_players+1)]
player_skills = [ random.randint(1,2800) for i in range(1,number_players+1)]

players = [Player(player_names[i],  player_skills[i], 1000) for i in range(number_players)]

In [ ]:
#a single simulation of 20 players and 100 games of chess throughout time
chess_game_rand(players, games) 

In [ ]:
#The graph of how each player's elo developed over the 100 games
x = range(games+1)     #+5
all_the_ys = [player_hist('p'+str(i),Match_history) for i in range(1,number_players+1)]
figure(figsize=(16, 6), dpi=80)

for i in range(len(all_the_ys)):   
    plt.plot(x, Conv(all_the_ys[i], games+1), label = "Player "+str(i+1))    #+5*[all_the_ys[i][-1]
plt.show()

In [ ]:
#How does the elo ranking compare to the real skill ranking?

print('Final ranking: '+ str([i.name for i in ranked(players)]))
print('Real ranking: ' + str([i.name for i in ranked_true(players)]) )

In [ ]:
#for montecarlo sim, we use the same players and run a bunch (100) of simulations of games
Lotta_games_rand = []

for i in range(monte_n):
    players = [Player(player_names[i],  player_skills[i], 1000) for i in range(number_players)]
    Lotta_games_rand += [chess_game_rand(players, games)]

In [ ]:
#For a single player , this is 100 simulations of how their 100 games (x axis) could have gone, we will average
#to find how well they're expected to do:

player_n = 1 #what player do we want to look at

x = range(games+1)  
all_the_lines = [player_hist('p'+str(player_n),Lotta_games_rand[i]) for i in range(monte_n)]
figure(figsize=(16, 6), dpi=80)


for i in range(len(all_the_lines)):
    plt.plot(x, all_the_lines[i],linewidth=1, color='yellow') 
    
plt.plot(x, [summ_i(all_the_lines, x) for x in range(games+1)], label = "Player "+str(player_n), color='red', linewidth=2 )





plt.legend()
plt.show()

In [ ]:
#we can now graph all players after doing montecarlo

x = range(games+1) 
figure(figsize=(16, 6), dpi=80) 

final_score_mcrlo=[]

for playerr in range(1, number_players+1):
    player_n = playerr #what player do we want to look at
    all_the_lines = [player_hist('p'+str(player_n),Lotta_games_rand[i]) for i in range(monte_n)]
    y_avgi = [summ_i(all_the_lines, x) for x in range(games+1)]
    final_score_mcrlo += [[playerr, y_avgi[-1]]]
    plt.plot(x,y_avgi , label = "Player "+str(player_n) , marker='o')

    
plt.legend()
plt.show()

In [ ]:
#real rankings vs expected (average) rankings of players:

#print('Real ranking: ' + str([i.name for i in ranked_true(players)]) )
#print('Average ranking: ' +str(['p'+str(i[0]) for i in Sort_mcrlo(final_score_mcrlo)]) )

for r in range(number_players):
    if [i.name for i in ranked_true(players)][r] == ['p'+str(i[0]) for i in Sort_mcrlo(final_score_mcrlo)][r]:
        print(Fore.GREEN + str(r), [i.name for i in ranked_true(players)][r],['p'+str(i[0]) for i in Sort_mcrlo(final_score_mcrlo)][r] )
    else:
        print(Fore.RED + str(r), [i.name for i in ranked_true(players)][r],['p'+str(i[0]) for i in Sort_mcrlo(final_score_mcrlo)][r] )


In [ ]:
Lotta_games_elo = []

for i in range(monte_n):
    players = [Player(player_names[i],  player_skills[i], 1000) for i in range(number_players)]
    Lotta_games_elo += [chess_game_elo(players, games)]

    
#we can now graph all players after doing montecarlo

x = range(games+1) 
figure(figsize=(16, 6), dpi=80) 

final_score_mcrlo=[]

for playerr in range(1, number_players+1):
    player_n = playerr #what player do we want to look at
    all_the_lines = [player_hist('p'+str(player_n),Lotta_games_elo[i]) for i in range(monte_n)]
    y_avgi = [summ_i(all_the_lines, x) for x in range(games+1)]
    final_score_mcrlo += [[playerr, y_avgi[-1]]]
    plt.plot(x,y_avgi , label = "Player "+str(player_n) )

    
#plt.legend()
plt.show()

In [ ]:
#real rankings vs expected (average) rankings of players:

for r in range(number_players):
    if [i.name for i in ranked_true(players)][r] == ['p'+str(i[0]) for i in Sort_mcrlo(final_score_mcrlo)][r]:
        print(Fore.GREEN + str(r), [i.name for i in ranked_true(players)][r],['p'+str(i[0]) for i in Sort_mcrlo(final_score_mcrlo)][r] )
    else:
        print(Fore.RED + str(r), [i.name for i in ranked_true(players)][r],['p'+str(i[0]) for i in Sort_mcrlo(final_score_mcrlo)][r] )


In [ ]:
[player for player in players]